# Анализ графов в Python
Тетрадка посвящена работе с графами в питоне на примере библиотеки NetworkX.

## Обзор NetworkX

NetworkX --- это питонячая библиотека, предназначенная для создания, обработки и изучения сложных сетей (aka графов). В отличие от других библиотек, которые вы можете встретить в сети (igraph, graphviz etc), она полностью написана на Python, благодаря чему ставится через Anaconda и не требует предустановки других пакетов.

Материал сильно опирается на лекцию по [введению в NetworkX](https://www.cl.cam.ac.uk/~cm542/teaching/2010/stna-pdfs/stna-lecture8.pdf).

In [ ]:
import networkx as nx

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Пример создания простого графа:

In [ ]:
# Создаём заготовку графа, в которую будут добавляться вершины и рёбра.
g = nx.Graph()

Добавление вершин:

In [ ]:
# Вершину графа можно называть как строкой, так и числом.
g.add_node(1)
g.add_node('n')

# Добавление нескольких вершин сразу.
g.add_nodes_from([2, 3])

Добавление рёбер:

In [ ]:
# Указываем две вершины, которые надо соединить.
g.add_edge(1, 'n')

# Добавление нескольких рёбер между указанными вершинами.
g.add_edges_from([(1, 2), (1, 3)])

# Добавление сразу двух вершин.
# Создаются и новые вершины, и ребро между ними!
g.add_edge('a', 'b', weight=0.1)
g.add_edge('b', 'c', weight=1.5)
g.add_edge('a', 'c', weight=1)
g.add_edge('c', 'd', weight=0.2)
g.add_edge('c', 153, weight=1.2)

Визуализация (при повторном запуске ячейки картинка может меняться):

In [ ]:
nx.draw_networkx(g, node_color='lightblue')
plt.axis('off')

print('Graph nodes:', g.nodes())

Вывод кратчайшего (или минимизирующего затраты) пути от одной вершины до другой:

In [ ]:
# Без учёта весов.
print(nx.shortest_path(g, 'b', 'd'))

# С учётом весов: теперь за переход по каждому ребру вносится плата.
print(nx.shortest_path(g, 'b', 'd', weight='weight'))

Точно так же, как ребру приписывался численный параметр `weight`, вершине можно приписывать любые характеристики:

In [ ]:
g.add_node(
    'node',
    date='06.05.2018',
    any_name='some information'
)
g.node['node']

Т.е. каждый узел графа можно воспринимать как питонячий словарь `dict` с произвольными ключами и значениями.

Фактически весь граф это просто словарь, где ключ -- номер вершины, значение -- набор вершин, соседних к ней:

In [ ]:
g.adj

In [ ]:
g['c']

Можно вывести и другое традиционное представление графа в виде матрицы смежности:

In [ ]:
nx.adjacency_matrix(g).todense()

Получение числа вершин графа:

In [ ]:
g.number_of_nodes()

In [ ]:
len(g)

Числа рёбер:

In [ ]:
g.number_of_edges()

Совокупная информация:

In [ ]:
print(nx.info(g))

При работе с графами часто бывает полезным получить доступ ко всем вершинам / рёбрам, что позволяет сделать метод `.nodes()` / `.edges()`:

In [ ]:
g.nodes(data=True)

In [ ]:
g.edges(data=True)

In [ ]:
for node in g.nodes():
    print(node, g.degree(node))

---

Закодим простой граф.

Создайте граф, содержащий $20$ вершин с именами $0, 1, \ldots, 19$:

In [ ]:
# YOUR CODE

Добавьте рёбра, соединяющие вершины через одну ($0$ и $2$, $1$ и $3$, ..., $18$ и $0$, $19$ и $1$):

In [ ]:
# YOUR CODE

Нарисуйте полученный граф:

In [ ]:
# YOUR CODE

## Реальные данные

Считаем данные о станциях московского метрополитена в 2014 году:

In [ ]:
metro_data = pd.read_csv('metro_2014_pairwise.csv')

metro_data.head()

Формат такой: две станции записаны в одной строке, если между ними есть перегон.

Общее число станций:

In [ ]:
len(metro_data['Start station'].unique())

Загрузим данные в граф из подготовленной таблицы:

- вершинами будут названия станций
- ребро между двумя станциями проводится, если между станциями есть перегон

In [ ]:
metro_graph = nx.from_pandas_edgelist(metro_data, source='Start station', target='End station')

# Указываем, что направление перегона между станциями нас не интересует.
# (как правило, можем поехать в обе стороны)
metro_graph = nx.to_undirected(metro_graph)

print(nx.info(metro_graph))

Нарисуем граф и сохраним его во внешний файл (изменение размера картинки и сохранение делается средствами matplotlib'а):

In [ ]:
plt.figure(figsize=(40, 40)) 

# Можно поэкспериментировать со способом отрисовки: помимо draw_networkx есть такие:
# draw_circular, draw_spectral, draw_random, etc.
nx.draw_networkx(metro_graph, with_labels=True, node_color='white', node_size=500)

plt.savefig('metro_2014.png', bbox_inches='tight')

Все степени вершин доступны в поле `.degree`, которое похоже по структуре на словарь из пар Имя_вершины: Степень_вершины. Чтобы можно было работать, как со словарём, достаточно навесить преобразование `dict()`:

In [ ]:
dict(metro_graph.degree)

Постройте гистограмму степеней вершин. Подпишите оси и название графика.

In [ ]:
# YOUR CODE

Почему наибольшее число вершин имеет степень $2$?

Выведите станцию, которая соединена с наибольшим числом станций (т.е. вершина станции имеет наибольшую степень). Для этого понадобится пройтись циклом по всем парам вершина-степень и запомнить пару с максимальной степенью.

In [ ]:
# YOUR CODE

С помощью `pandas` выведите станции, на которые можно попасть напрямую от найденной (т.е. те, которые соединены с ней):

In [ ]:
# YOUR CODE

Библиотека NetworkX позволяет найти кратчайший путь между вершинами, по которому можно судить об их взаимном расположении:

In [ ]:
nx.shortest_path(metro_graph, 'Сокольническая_Библиотека_имени_Ленина', 'Серпуховско-Тимирязевская_Боровицкая')

In [ ]:
nx.shortest_path(metro_graph, 'Сокольническая_Библиотека_имени_Ленина', 'Серпуховско-Тимирязевская_Полянка')

А ещё можно вывести кратчайшие пути до всех вершин от данной:

In [ ]:
nx.single_source_shortest_path(metro_graph, 'Сокольническая_Библиотека_имени_Ленина')

Выделим некоторые кратчайшие пути цветом (при желании можно сохранить картинку, как и раньше).

In [ ]:
# Функция для выделения цветом путей, пример использования ниже.
# https://github.com/jtorrents/pydata_bcn_NetworkX/blob/master/NetworkX_SNA_workshop_with_solutions.ipynb
def plot_paths(G, paths):
    plt.figure(figsize=(36, 36))
    pos = nx.fruchterman_reingold_layout(G)
    nx.draw_networkx_nodes(G, pos=pos, node_size=4000, node_color='white')
    nx.draw_networkx_labels(G, pos=pos, labels={n: n for n in G})
    # Draw edges
    nx.draw_networkx_edges(G, pos=pos)
    for path in paths:
        edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos=pos, edgelist=edges, edge_color='red', width=3)
    ax = plt.gca()
    ax.set_axis_off()
    ax.grid(None)

In [ ]:
# Вывод кратчайших путей из одной вершины до всех остальных.
plot_paths(metro_graph, nx.single_source_shortest_path(metro_graph, 'Сокольническая_Библиотека_имени_Ленина').values())

In [ ]:
# Вывод нескольких кратчайших путей между конкретными парами вершин.
plot_paths(
    metro_graph,
    [
        nx.shortest_path(metro_graph, 'Сокольническая_Библиотека_имени_Ленина', 'Серпуховско-Тимирязевская_Алтуфьево'),
        nx.shortest_path(metro_graph, 'Сокольническая_Сокольники', 'Калужско-Рижская_Третьяковская')
    ]
)

### Bonus task: раскрашиваем граф метро
Конечно, можно нарисовать карту метро вот так:

In [ ]:
plt.figure(figsize=(16, 16)) 

nx.draw_networkx(metro_graph, with_labels=False)

А можно каждую вершину подкрасить цветом соответствующей ветки!

Для этого сначала создадим словарь с названиями цветов для каждой ветки (при желании поменяйте цвета на своё усмотрение, таблица с matplotlib'овскими цветами [тут](matplotlib.org/examples/color/named_colors)):

In [ ]:
line_colors = {
    'Сокольническая': 'red',
    'Замоскворецкая': 'green',
    'Арбатско-Покровская': 'darkblue',
    'Филевская': 'blue',
    'Кольцевая': 'saddlebrown',
    'Калужско-Рижская': 'orangered',
    'Таганско-Краснопресненская': 'blueviolet',
    'Калининская': 'gold',
    'Серпуховско-Тимирязевская': 'gray',
    'Люблинская': 'lawngreen',
    'Каховская': 'palegreen',
    'Бутовская': 'lightsteelblue',
}

Ниже приведена функция, которая принимает название станции в формате `Имя-ветки_Имя-станции` (именно в таком формате хранятся названия в нашей таблице) и словарь вида `Имя-ветки: Цвет` (уже определённый `line_colors`). Функция возвращает цвет станции в соответствии с веткой, на которой она расположена.

In [ ]:
def return_station_color(station, line_colors):
    for line in line_colors:
        if station.startswith(line):
            return line_colors[line]
    print('Something went wrong: couldn\'t find the line for station {}.\nReturn black color.'.format(station))
    return 'black'

Пример использования:

In [ ]:
return_station_color('Калининская_Третьяковская', line_colors)

In [ ]:
return_station_color('Третьяковская', line_colors)

Создайте список цветов для наших вершин.

Для этого надо в цикле пройтись по всем вершинам в `metro_graph.nodes()` и добавить в список их цвета:

In [ ]:
station_colors = []

# YOUR CODE

In [ ]:
plt.figure(figsize=(16, 16)) 

# Опять же, можно поиграть со способами отрисовки.
nx.draw_networkx(metro_graph, with_labels=False, node_color=station_colors)